In [ ]:
!pip uninstall -y numpy scipy albumentations scikit-image
!pip install numpy==1.26.4 scipy==1.11.4 albumentations==1.3.1 scikit-image==0.21.0


In [1]:
pip install ultralytics --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

path = "/kaggle/input/yv9elastwiehts/last_pt_where_stop/weights/last (21).pt"
print("Exists?" , os.path.exists(path))


Exists? True


In [3]:
# =========================================================
# YOLOv9-E (ELAN) — Resume CCTV Training (Weapons + Violence)
# =========================================================

import os
import random
import numpy as np
import torch
from ultralytics import YOLO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics.data.augment import Albumentations as UAlbumentations
import shutil

# -------------------------
# USER CONFIG
# -------------------------
DATA_YAML = "/kaggle/input/violenceweapondetection/data.yaml"
PROJECT_DIR = "/kaggle/working/yolov9e_cctv_runs"
RUN_NAME = "yolov9e_cctv_elan"

# ✅ Use previous checkpoint instead of pretrained model
CHECKPOINT = "/kaggle/input/yv9elastwiehts/last_pt_where_stop/weights/last (21).pt"

SEED = 42
EPOCHS = 200
IMGSZ = 640
BATCH = 8
WORKERS = 2
LR0 = 3e-4
WEIGHT_DECAY = 4e-4
MOSAIC = 0.15
MIXUP = 0.08
AMP = True
CACHE = False

# -------------------------
# Seed Setup
# -------------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True

set_seed(SEED)
print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())

# -------------------------
# Albumentations CCTV-safe augmentations
# -------------------------
def custom_cctv_aug():
    return A.Compose([
        A.Resize(height=IMGSZ, width=IMGSZ, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.96, 1.04), rotate=(-4, 4), shear=(-2, 2), p=0.4),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.4),
        A.RandomGamma(gamma_limit=(90, 110), p=0.25),
        A.HueSaturationValue(hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.25),
        A.CLAHE(clip_limit=2.0, tile_grid_size=(8,8), p=0.15),
        A.GaussianBlur(blur_limit=(3,3), p=0.05),
        ToTensorV2()
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.1))

# -------------------------
# Model setup (Resume Checkpoint)
# -------------------------
model = YOLO(CHECKPOINT)
model.add_callback('on_preprocess_batch', UAlbumentations(custom_cctv_aug()))

# -------------------------
# ✅ Resume Training (all params kept same)
# -------------------------
model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0,
    workers=WORKERS,
    optimizer='AdamW',
    lr0=LR0,
    weight_decay=WEIGHT_DECAY,
    warmup_epochs=2,
    patience=25,
    cos_lr=True,
    amp=AMP,
    cache=CACHE,
    rect=False,
    mosaic=MOSAIC,
    mixup=MIXUP,
    project=PROJECT_DIR,
    name=RUN_NAME,
    exist_ok=True,
    seed=SEED,
    freeze=0,
    save=True,
    resume=True      # ✅ THE MOST IMPORTANT LINE
)

# -------------------------
# Validation + Export
# -------------------------
print("\n==> Final validation...")
metrics = model.val()
print(metrics)

print("\n==> Exporting best model (ONNX)")
model.export(format="onnx", dynamic=True)

# -------------------------
# Zip results for download
# -------------------------
out_zip = f"/kaggle/working/{RUN_NAME}.zip"
shutil.make_archive(out_zip.replace('.zip',''), 'zip', os.path.join(PROJECT_DIR, RUN_NAME))
print(f"\n✅ Zipped run to: {out_zip}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Torch: 2.6.0+cu124 CUDA: True
albumentations: ImageCompression.__init__() got an unexpected keyword argument 'quality_range'
Ultralytics 8.3.226 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/input/violenceweapondetection/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud

In [ ]:
import shutil

# Source folder you want to zip
source_dir = "/kaggle/working/yolov9e_cctv_runs"

# Output zip file path (without .zip extension)
output_zip = "/kaggle/working/yolov9e_cctv_runs_zip"

# Create zip
shutil.make_archive(output_zip, 'zip', source_dir)

print("Zip file created at:", output_zip + ".zip")


In [ ]:
!pip install numpy==1.25.2 --upgrade
!pip install scipy==1.11.2 --upgrade


In [ ]:
import albumentations as A
print(A.__version__)


In [ ]:
pip uninstall -y numpy


In [ ]:
pip install numpy==1.26.4


In [ ]:
import cv2
import numpy as np
import albumentations as A

print("✅ OpenCV:", cv2.__version__)
print("✅ NumPy:", np.__version__)
print("✅ Albumentations:", A.__version__)


In [ ]:
!pip install ultralytics

In [ ]:
import os
import cv2
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

print("✅ Albumentations", A.__version__)
print("✅ Torch", torch.__version__)

# =========================================================
# ✅ Custom CCTV Augmentation Pipeline (without Mosaic/MixUp)
# =========================================================
train_transform = A.Compose([
    # Geometric transformations
    A.RandomResizedCrop(size=(640, 640), scale=(0.7, 1.0), ratio=(0.75, 1.33), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15,
                       border_mode=cv2.BORDER_CONSTANT, p=0.8),

    # Brightness / contrast / color (CCTV-like lighting)
    A.RandomBrightnessContrast(p=0.7),
    A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=15, p=0.6),
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.4),
    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.3),

    # Blur / noise (simulate poor CCTV quality)
    A.MotionBlur(p=0.3),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    A.GaussNoise(var_limit=(5, 25), p=0.3),

    # Dropouts (simulate occlusion or fog)
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.4),

    ToTensorV2()
])

# =========================================================
# ✅ Load Pretrained YOLOv8m Model
# =========================================================
model = YOLO("yolov8m.pt")

# Disable YOLO’s default heavy augmentations but keep light mosaic/mixup
model.overrides['augment'] = True     # Keep augment True so it can still do small mosaic/mixup
model.overrides['rect'] = False
model.overrides['mosaic'] = 0.3       # Small mosaic probability
model.overrides['mixup'] = 0.2        # Small mixup probability
model.overrides['copy_paste'] = 0.0   # Disable copy-paste

# =========================================================
# ✅ Dataset Paths
# =========================================================
data_yaml = "/kaggle/input/violenceweapondetection/data.yaml"  # replace with your dataset yaml
save_dir = "/kaggle/working/yolov8m_cctv_aug_runs"


# =========================================================
# ✅ Hook Albumentations into YOLO dataloader
# =========================================================
def custom_dataloader_hook(dataset):
    """Inject Albumentations transforms into YOLO’s training dataloader."""
    dataset.transforms = train_transform
    return dataset

# Apply Albumentations augmentations AFTER dataloader is ready
model.add_callback("on_fit_epoch_start", lambda trainer: custom_dataloader_hook(trainer.train_loader.dataset))

# =========================================================
# ✅ Train Model
# =========================================================
results = model.train(
    data=data_yaml,
    epochs=150,                # maximum number of epochs
    imgsz=640,
    batch=8,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=save_dir,
    name="yolov8m_cctv_aug",
    workers=2,
    lr0=0.001,
    optimizer="SGD",
    pretrained=True,
    verbose=True,
    val=True,
    exist_ok=True,
    patience=15              # stops training if no improvement for 25 epochs
)

# =========================================================
# ✅ Evaluate Metrics
# =========================================================
metrics = model.val()
print("\n📊 Validation Metrics:")
try:
    print(f"Precision: {metrics.box.pr:.4f}")
    print(f"Recall: {metrics.box.re:.4f}")
    print(f"mAP50: {metrics.box.map50:.4f}")
    print(f"mAP50-95: {metrics.box.map:.4f}")
except Exception as e:
    print("⚠️ Metrics output error:", e)
    print(metrics)

print("\n✅ Fine-tuning completed successfully with CCTV augmentations (small mosaic & mixup)!")


In [ ]:
import shutil

shutil.make_archive(
    "/kaggle/working/yolov8m_final_cctv_aug_runs",   # output zip path (without .zip extension)
    'zip',                                     # format
    "/kaggle/working/yolov8m_cctv_aug_runs"  # folder to zip
)
